In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [2]:
df = pd.read_csv('SCMS_Delivery_History_Dataset.csv')

In [3]:
df.head()

,ID,Project Code,PQ #,PO / SO #,ASN/DN #,Country,Managed By,Fulfill Via,Vendor INCO Term,Shipment Mode,...,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,First Line Designation,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,...,30,19,551.0,29.00,0.97,Ranbaxy Fine Chemicals LTD,Yes,13,780.34,NaN
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,...,240,1000,6200.0,6.20,0.03,"Aurobindo Unit III, India",Yes,358,4521.5,NaN
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,...,100,500,40000.0,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,Yes,171,1653.78,NaN
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,31920,127360.8,3.99,0.07,"Ranbaxy, Paonta Shahib, India",Yes,1855,16007.06,NaN
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,38000,121600.0,3.20,0.05,"Aurobindo Unit III, India",Yes,7590,45450.08,NaN


In [4]:
df.shape

(10324, 33)

In [5]:
df.columns

Index(['ID', 'Project Code', 'PQ #', 'PO / SO #', 'ASN/DN #', 'Country',
       'Managed By', 'Fulfill Via', 'Vendor INCO Term', 'Shipment Mode',
       'PQ First Sent to Client Date', 'PO Sent to Vendor Date',
       'Scheduled Delivery Date', 'Delivered to Client Date',
       'Delivery Recorded Date', 'Product Group', 'Sub Classification',
       'Vendor', 'Item Description', 'Molecule/Test Type', 'Brand', 'Dosage',
       'Dosage Form', 'Unit of Measure (Per Pack)', 'Line Item Quantity',
       'Line Item Value', 'Pack Price', 'Unit Price', 'Manufacturing Site',
       'First Line Designation', 'Weight (Kilograms)', 'Freight Cost (USD)',
       'Line Item Insurance (USD)'],
      dtype='object')

In [6]:
for item in df.columns:
    print(item,':',df[item].nunique())

ID : 10324
Project Code : 142
PQ # : 1237
PO / SO # : 6233
ASN/DN # : 7030
Country : 43
Managed By : 4
Fulfill Via : 2
Vendor INCO Term : 8
Shipment Mode : 4
PQ First Sent to Client Date : 765
PO Sent to Vendor Date : 897
Scheduled Delivery Date : 2006
Delivered to Client Date : 2093
Delivery Recorded Date : 2042
Product Group : 5
Sub Classification : 6
Vendor : 73
Item Description : 184
Molecule/Test Type : 86
Brand : 48
Dosage : 54
Dosage Form : 17
Unit of Measure (Per Pack) : 31
Line Item Quantity : 5065
Line Item Value : 8741
Pack Price : 1175
Unit Price : 183
Manufacturing Site : 88
First Line Designation : 2
Weight (Kilograms) : 4688
Freight Cost (USD) : 6733
Line Item Insurance (USD) : 6722


In [7]:
'''
2 types Fulfill Via - From RDC(52%) and Direct Drop(48%)
INCO Terms - N/A for RDC, 7 Inco Terms used- mostly EXW(56%),DDP(29%),FCA(8%),CIP(5.5%)
4 Shipment modes - Air (59%), Truck (27%), Air Charter (6%), Ocean (3%), Nan (3%)
PQ First Sent Date - NA (24%)
PO Sent to Vendor Date - NA for RDC + Not captured (55%)
5 Product Group - ARV (83%), HRDT(17%), 3 negligible
6 Sub Class - Adult (64%), Pediatric (19%), HIV test(15%), HIV-Ancillary (2%), Malaria and ACT negligible
72 Non-RDC Vendors
48 Brands
17 Dosage Form
5065 Line Item Quantity
88 Manufacturing Sites
First Line Designation - Yes (68%), No(32%)
Weight - Not defined for 40% data
Freight Cost - Not clear for 41% data
Insurance NAN for 3%
'''
df['Freight Cost (USD)'].value_counts(normalize=True,dropna=False) #[df['Vendor INCO Term'] == 'N/A - From RDC']

Freight Included in Commodity Cost    0.139675
Invoiced Separately                   0.023150
9736.1                                0.003487
6147.18                               0.002615
13398.06                              0.001550
                                        ...   
749.36                                0.000097
7060.97                               0.000097
7616.19                               0.000097
12793.7                               0.000097
See DN-4282 (ID#:83919)               0.000097
Name: Freight Cost (USD), Length: 6733, dtype: float64

In [8]:
df.isnull().sum()
#360 rows do not have Shipment Mode which we aim to predict

ID                                 0
Project Code                       0
PQ #                               0
PO / SO #                          0
ASN/DN #                           0
Country                            0
Managed By                         0
Fulfill Via                        0
Vendor INCO Term                   0
Shipment Mode                    360
PQ First Sent to Client Date       0
PO Sent to Vendor Date             0
Scheduled Delivery Date            0
Delivered to Client Date           0
Delivery Recorded Date             0
Product Group                      0
Sub Classification                 0
Vendor                             0
Item Description                   0
Molecule/Test Type                 0
Brand                              0
Dosage                          1736
Dosage Form                        0
Unit of Measure (Per Pack)         0
Line Item Quantity                 0
Line Item Value                    0
Pack Price                         0
U

In [9]:
df['Shipment Mode'].value_counts(normalize=True,dropna=False)

Air            0.592115
Truck          0.274119
Air Charter    0.062960
Ocean          0.035936
NaN            0.034870
Name: Shipment Mode, dtype: float64

In [10]:
df['Country'].value_counts(normalize=True,dropna=False)

South Africa          0.136188
Nigeria               0.115653
Côte d'Ivoire         0.104901
Uganda                0.075455
Vietnam               0.066641
Zambia                0.066157
Haiti                 0.063444
Mozambique            0.061120
Zimbabwe              0.052112
Tanzania              0.050271
Rwanda                0.041651
Congo, DRC            0.032255
Guyana                0.022956
Ethiopia              0.020922
South Sudan           0.015885
Kenya                 0.010752
Burundi               0.009492
Namibia               0.009202
Cameroon              0.007265
Botswana              0.006780
Ghana                 0.005618
Dominican Republic    0.005037
Sudan                 0.004456
Swaziland             0.003390
Mali                  0.001647
Guatemala             0.001453
Pakistan              0.001453
Malawi                0.001356
Benin                 0.001259
Lebanon               0.000775
Libya                 0.000775
Angola                0.000678
Liberia 

In [11]:
df.dtypes

ID                                int64
Project Code                     object
PQ #                             object
PO / SO #                        object
ASN/DN #                         object
Country                          object
Managed By                       object
Fulfill Via                      object
Vendor INCO Term                 object
Shipment Mode                    object
PQ First Sent to Client Date     object
PO Sent to Vendor Date           object
Scheduled Delivery Date          object
Delivered to Client Date         object
Delivery Recorded Date           object
Product Group                    object
Sub Classification               object
Vendor                           object
Item Description                 object
Molecule/Test Type               object
Brand                            object
Dosage                           object
Dosage Form                      object
Unit of Measure (Per Pack)        int64
Line Item Quantity                int64


In [12]:
#Convering dates into datetime format. For 'PQ First Sent to Client Date' ,'PO Sent to Vendor Date ' . Coerce the errors as some of the dates are not defined.
dt = ['PQ First Sent to Client Date' ,'PO Sent to Vendor Date','Scheduled Delivery Date','Delivered to Client Date', 'Delivery Recorded Date']
for col in dt:
    df[col] = pd.to_datetime(df[col], errors = 'coerce')

### Filling Nan Values
1. Shipment Mode - Drop rows
2. Dosage - Filling by mode
3. Line Item Insurance - Mean Percentage of Line Item Insurance/Line Item Value
4. PQ date - Reverse calculation by subtracting average days from Schedule Delivery to First Price Quotation
5. PO Date - Reverse calculation by subtracting average days from Schedule Delivery to Purchase Order
6. Weights - where ID is mentioned. Taken same as mentioned ID. Remaining filled with mean.
7. Freight - 0 where included in Commodity Cost, where ID is mentioned. Taken same as mentioned ID. Remaining filled with mean.

In [13]:
#  Create a copy of data
df2 = df.copy()

In [14]:
# Drop rows with no shipment mode
missing_shipment = df[df['Shipment Mode'].isna()].index
df = df.drop(missing_shipment, axis=0).reset_index(drop= True)

In [15]:
# Replace NAN with mode in Dosage column
df['Dosage'] = df['Dosage'].fillna(df['Dosage'].mode()[0])

In [16]:
# Fill Insurance with percentage value of line item value
perc = df['Line Item Insurance (USD)'].sum() / df['Line Item Value'][df['Line Item Insurance (USD)'] >= 0].sum()
df['Line Item Insurance (USD)'] = df['Line Item Insurance (USD)'].fillna(round(df['Line Item Value']*perc, 2))

In [17]:
# Fill PO & PQ dates
#Calculate Average days between Price Quote-->Purchase Order--> Scheduled Delivery
pq_del_days = round((df['Scheduled Delivery Date'] - df['PQ First Sent to Client Date']).dt.days.mean(),0)
pq_po_days = round((df['PO Sent to Vendor Date'] - df['PQ First Sent to Client Date']).dt.days.mean(),0)
po_del_days = round((df['Scheduled Delivery Date'] - df['PO Sent to Vendor Date']).dt.days.mean(),0)

In [18]:
print (pq_del_days)
print (pq_po_days)
print (po_del_days)

172.0
54.0
106.0


In [19]:
# Assigning estimated dates of Price Quotation and Purchase Order
df['PQ First Sent to Client Date'] = df['PQ First Sent to Client Date'].fillna(df['Scheduled Delivery Date'] - timedelta(days=pq_del_days))
df['PO Sent to Vendor Date'] = df['PO Sent to Vendor Date'].fillna(df['Scheduled Delivery Date'] - timedelta(days=po_del_days))

In [20]:
# Tackling Weight & Freight missing values
df['Freight Cost (USD)'] = df['Freight Cost (USD)'].replace('Freight Included in Commodity Cost',0)

In [21]:
# Dictionaries with ID as keys and weight/freight as values
id_weight = dict(zip(df['ID'],df['Weight (Kilograms)']))
id_freight = dict(zip(df['ID'],df['Freight Cost (USD)']))

In [22]:
# Functions to get weight and freight from corresponding ID numbers
def get_weight(data):
    data = pd.to_numeric(data, errors = 'ignore')
    if type(data) == str:
        try:
            d = data.split(':')
            e = d[1][:-1]
            return id_weight.get(int(e))
        except:
            return (data)
    else:
        return (data)
    
def get_freight(data):
    data = pd.to_numeric(data, errors = 'ignore')
    if type(data) == str:
        try:
            d = data.split(':')
            e = d[1][:-1]
            return id_freight.get(int(e))
        except:
            return (data)
    else:
        return (data)

In [23]:
df['Weight (Kilograms)'] = df['Weight (Kilograms)'].apply(get_weight)
df['Freight Cost (USD)'] = df['Freight Cost (USD)'].apply(get_freight)

In [24]:
df['Weight (Kilograms)'] = pd.to_numeric(df['Weight (Kilograms)'], errors = 'coerce')
df['Freight Cost (USD)'] = pd.to_numeric(df['Freight Cost (USD)'], errors = 'coerce')

In [25]:
df['Weight (Kilograms)'] = df['Weight (Kilograms)'].fillna(df['Weight (Kilograms)'].mean())
df['Freight Cost (USD)'] = df['Freight Cost (USD)'].fillna(df['Freight Cost (USD)'].mean())

In [26]:
df.isnull().sum()

ID                              0
Project Code                    0
PQ #                            0
PO / SO #                       0
ASN/DN #                        0
Country                         0
Managed By                      0
Fulfill Via                     0
Vendor INCO Term                0
Shipment Mode                   0
PQ First Sent to Client Date    0
PO Sent to Vendor Date          0
Scheduled Delivery Date         0
Delivered to Client Date        0
Delivery Recorded Date          0
Product Group                   0
Sub Classification              0
Vendor                          0
Item Description                0
Molecule/Test Type              0
Brand                           0
Dosage                          0
Dosage Form                     0
Unit of Measure (Per Pack)      0
Line Item Quantity              0
Line Item Value                 0
Pack Price                      0
Unit Price                      0
Manufacturing Site              0
First Line Des

In [27]:
df.to_csv('stage1_process.csv', index = False)

In [28]:
#df = pd.read_csv('stage1_process.csv')

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9964 entries, 0 to 9963
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   ID                            9964 non-null   int64         
 1   Project Code                  9964 non-null   object        
 2   PQ #                          9964 non-null   object        
 3   PO / SO #                     9964 non-null   object        
 4   ASN/DN #                      9964 non-null   object        
 5   Country                       9964 non-null   object        
 6   Managed By                    9964 non-null   object        
 7   Fulfill Via                   9964 non-null   object        
 8   Vendor INCO Term              9964 non-null   object        
 9   Shipment Mode                 9964 non-null   object        
 10  PQ First Sent to Client Date  9964 non-null   datetime64[ns]
 11  PO Sent to Vendor Date        

In [30]:
date_columns = ['PQ First Sent to Client Date' ,'PO Sent to Vendor Date','Scheduled Delivery Date','Delivered to Client Date']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors = 'raise')

In [31]:
# Creating feature for number of days delay in delivery
df['Delay'] = round((df['Delivered to Client Date'] - df['Scheduled Delivery Date']).dt.days,0)

In [32]:
# Creating feature for Total Cost of Shipment
df['Total Cost'] = df['Line Item Value'] + df['Freight Cost (USD)'] + df['Line Item Insurance (USD)']

In [33]:
# Creating separate columns for date month and year
date_columns = ['PQ First Sent to Client Date' ,'PO Sent to Vendor Date','Scheduled Delivery Date','Delivered to Client Date']
for column in date_columns:
    df[column] = pd.to_datetime(df[column])
    df[column + ' year'] = df[column].apply(lambda x: x.year)
    df[column + ' month'] = df[column].apply(lambda x: x.month)
    df[column + ' day'] = df[column].apply(lambda x: x.day)

In [34]:
df.columns

Index(['ID', 'Project Code', 'PQ #', 'PO / SO #', 'ASN/DN #', 'Country',
       'Managed By', 'Fulfill Via', 'Vendor INCO Term', 'Shipment Mode',
       'PQ First Sent to Client Date', 'PO Sent to Vendor Date',
       'Scheduled Delivery Date', 'Delivered to Client Date',
       'Delivery Recorded Date', 'Product Group', 'Sub Classification',
       'Vendor', 'Item Description', 'Molecule/Test Type', 'Brand', 'Dosage',
       'Dosage Form', 'Unit of Measure (Per Pack)', 'Line Item Quantity',
       'Line Item Value', 'Pack Price', 'Unit Price', 'Manufacturing Site',
       'First Line Designation', 'Weight (Kilograms)', 'Freight Cost (USD)',
       'Line Item Insurance (USD)', 'Delay', 'Total Cost',
       'PQ First Sent to Client Date year',
       'PQ First Sent to Client Date month',
       'PQ First Sent to Client Date day', 'PO Sent to Vendor Date year',
       'PO Sent to Vendor Date month', 'PO Sent to Vendor Date day',
       'Scheduled Delivery Date year', 'Scheduled Delivery 

In [35]:
# Dropping columns which won't add value or would make the data unmanageable
df = df.drop(['ID', 'PQ #','PO / SO #','ASN/DN #', 'Managed By','PQ First Sent to Client Date', 'PO Sent to Vendor Date',
       'Scheduled Delivery Date', 'Delivered to Client Date',
       'Delivery Recorded Date','Item Description', 'Molecule/Test Type',], axis = 1)

In [36]:
# Renaming Columns
df.columns = ['proj_code','country', 'via',
       'inco', 'ship_mode', 'prod_grp',
       'sub_class', 'vendor', 'brand', 'dosage', 'dosage_form',
       'per_pack_qty', 'line_qty', 'line_value',
       'pack_price', 'unit_price', 'manu_site',
       'first_line', 'weight', 'freight',
       'insurance', 'delay', 'total_cost',
       'quote_year',
       'quote_month',
       'quote_day', 'po_year',
       'po_month', 'po_day',
       'sch_del_year', 'sch_del_month',
       'sch_del_day', 'actual_del_year',
       'actual_del_month', 'actual_del_day']

In [37]:
df.describe()

,per_pack_qty,line_qty,line_value,pack_price,unit_price,weight,freight,insurance,delay,total_cost,...,quote_day,po_year,po_month,po_day,sch_del_year,sch_del_month,sch_del_day,actual_del_year,actual_del_month,actual_del_day
count,9964.000000,9964.000000,9.964000e+03,9964.000000,9964.000000,9964.000000,9964.000000,9964.000000,9964.000000,9.964000e+03,...,9964.000000,9964.000000,9964.000000,9964.000000,9964.000000,9964.000000,9964.000000,9964.000000,9964.000000,9964.000000
mean,77.425733,18612.925632,1.592953e+05,21.734272,0.590277,4516.521007,10900.582315,239.313809,-6.150442,1.704352e+05,...,15.566038,2011.115717,6.512746,15.590325,2011.402047,6.493878,17.774488,2011.391008,6.477318,16.143818
std,76.393988,40572.830601,3.493419e+05,45.710003,2.292690,12434.759618,17130.299639,500.186015,27.396541,3.565207e+05,...,8.687185,2.390670,3.477655,8.613145,2.402686,3.288049,9.404055,2.411718,3.326912,8.794376
min,1.000000,1.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,-372.000000,2.500000e+00,...,1.000000,2006.000000,1.000000,1.000000,2006.000000,1.000000,1.000000,2006.000000,1.000000,1.000000
25%,30.000000,405.000000,4.272032e+03,4.120000,0.080000,404.000000,1374.732500,6.530000,-4.000000,9.522917e+03,...,8.000000,2009.000000,3.000000,9.000000,2009.000000,4.000000,10.000000,2009.000000,4.000000,9.000000
50%,60.000000,3035.000000,3.042884e+04,8.970000,0.160000,2156.000000,5530.110000,46.960000,0.000000,4.034555e+04,...,15.000000,2011.000000,7.000000,16.000000,2012.000000,7.000000,18.000000,2012.000000,6.000000,16.000000
75%,90.000000,17441.750000,1.686673e+05,23.260000,0.460000,4516.521007,13290.310000,251.730000,0.000000,1.837360e+05,...,23.000000,2013.000000,10.000000,22.000000,2014.000000,9.000000,27.000000,2014.000000,9.000000,24.000000
max,1000.000000,619999.000000,5.951990e+06,1345.640000,41.680000,857354.000000,289653.200000,7708.440000,192.000000,6.027871e+06,...,31.000000,2015.000000,12.000000,31.000000,2015.000000,12.000000,31.000000,2015.000000,12.000000,31.000000


In [38]:
df.to_csv('pre_encoding_data.csv', index = False)